<a href="https://colab.research.google.com/github/khaouitiabdelhakim/llm_fine_tuning/blob/main/diabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r llm_fine_tuning

rm: cannot remove 'llm_fine_tuning': No such file or directory


In [ ]:
!git clone https://github.com/khaouitiabdelhakim/llm_fine_tuning

Cloning into 'llm_fine_tuning'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 52 (delta 16), reused 39 (delta 8), pack-reused 0
Receiving objects: 100% (52/52), 6.35 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
!pip install -r ./llm_fine_tuning/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 48.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from llm_fine_tuning.Tunner import fine_tunner

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
df = pd.read_csv("./llm_fine_tuning/Data/train_samples.csv")[:100]
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
tuner = fine_tunner.Llama_Tuner()

Mounted at /content/drive


In [ ]:
train_data = tuner.to_dataset(list(df['question']), list(df['answer']))

i = 0
for _ in train_data:
  print(_)
  i += 1
  if i >= 10:
    break

{'text': "<s>[INST] What is (are) Glaucoma ? [/INST] Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Op

In [ ]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'
save_name = './drive/MyDrive/fine-tuned-model'

quant_config = {
    'load_in_4bit': True,
    'bnb_4bit_quant_type': "nf4",
    'bnb_4bit_compute_dtype': getattr(torch, "float16"),
    'bnb_4bit_use_double_quant': False
}

lora_config = {
    'lora_alpha': 16,
    'lora_dropout': 0.1,
    'r': 64,
    'bias': "none",
    'task_type': "CASUAL_LM",
    # use None if you dont wat to use target modules.
    'target_modules': [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ]
}

train_config = {
    'output_dir': "./results",
    'num_train_epochs': 1,
    'per_device_train_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "paged_adamw_32bit",
    'save_steps': 25,
    'logging_steps': 25,
    'learning_rate': 2e-4,
    'weight_decay': 0.001,
    'fp16': False,
    'bf16': False,
    'max_grad_norm': 0.3,
    'max_steps': -1,
    'warmup_ratio': 0.03,
    'group_by_length': True,
    'lr_scheduler_type': "constant",
    'report_to': "tensorboard",
    'remove_unused_columns': False
}

In [ ]:
tuner.load_model(model_name=model_name, quant_config=quant_config, lora_config=lora_config, train_config=train_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
tuner.tune_and_save(train_data, save_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.410700


In [ ]:
prompt = "Who are you?"
tuner.generate_base_text(prompt)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences betwee

'<s>[INST] Who are you? [/INST] I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner.[INST] What is your purpose? [/INST] I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. [INST] What is your purpose? [/INST] I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner.'

In [ ]:
prompt = "Who are you?"
tuner.generate_text(prompt)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'Rober

'<s>[INST] Who are you? [/INST] I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. everybody is talking about me. I am the latest AI assistant to hit the market, and I am already making waves. I am the first AI assistant that can understand and respond to human input in a conversational manner. I am the latest AI assistant to hit the market, and I am already making waves. I am the first AI assistant that can understand and respond to human input in a conversational manner. I am the latest AI assistant to hit the market, and I am already making waves. I am the first AI assistant that can understand and respond to human input in a conversational manner. I am the latest AI assistant to hit the market, and I am already making waves. I am the first AI assistant that can understand and respond to human'

In [ ]:
test_df = pd.read_csv("./llm_fine_tuning/Data/test.csv")[:2]
test_dataset = tuner.to_dataset(list(test_df['question']), list(test_df['answer']))

reference = []
for item in test_dataset:
  reference.append(item['text'])

response = []
base_response = []
for ques in tqdm(test_df['question']):
  base_resp = tuner.generate_base_text(ques)
  resp = tuner.generate_text(ques)
  base_response.append(base_resp.strip())
  response.append(resp.strip())

  base_df = pd.DataFrame(base_response, columns=['response'])
  base_df.to_csv(f"./drive/MyDrive/{model_name}.csv", index=False)

  tuned_df = pd.DataFrame(response, columns=['response'])
  tuned_df.to_csv(f"./drive/MyDrive/{model_name}_tuned.csv", index=False)

bleu_score = tuner.eval_model(reference, base_response, ['bleu'])
print(f"The bleu score for base model is {bleu_score}")

bleu_score = tuner.eval_model(reference, response, ['bleu'])
print(f"The bleu score for tuned model is {bleu_score}")

rouge_score = tuner.eval_model(reference, base_response, ['rouge'])
print(f"The rouge score for base model is {bleu_score}")

rouge_score = tuner.eval_model(reference, response, ['rouge'])
print(f"The rouge score for tuned model is {bleu_score}")

rouge_score = tuner.eval_model(reference, base_response, ['ter'])
print(f"The ter score for base model is {bleu_score}")

rouge_score = tuner.eval_model(reference, response, ['ter'])
print(f"The ter score for tuned model is {bleu_score}")

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM',

OSError: Cannot save file into a non-existent directory: 'drive/MyDrive/NousResearch'

In [ ]:
# For Testing

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Check if CUDA is available
print(torch.cuda.is_available())

# Path to your fine-tuned adapter model
adapter_path = './content/results/checkpoint-25'

# Load the base model
base_model_name = 'NousResearch/Llama-2-7b-chat-hf'  # Change this to the original base model you fine-tuned

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load the adapter configuration
adapter_config = PeftConfig.from_pretrained(adapter_path)

# Load the model with the adapter
model = PeftModel.from_pretrained(base_model, adapter_path)

# Check if the model is loaded on the GPU, if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate text based on a prompt
def generate_text(prompt, max_length=50, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id  # Ensure the model doesn't generate more than max_length tokens
    )
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

# Example usage
prompt = "What can you do?"
generated_texts = generate_text(prompt)
for i, text in enumerate(generated_texts):
    print(f"Generated text {i + 1}: {text}")


ModuleNotFoundError: No module named 'peft'